In [1]:
import pandas as pd
from pathlib import Path
import pylatex as pl
from pylatex.utils import NoEscape
import pymc as pm
import arviz as az
import plotly.express as px
import numpy as np
import pandas as pd

from estival.model import BayesianCompartmentalModel
import estival.priors as esp
import estival.targets as est
from estival.wrappers import pymc as epm
from tbdynamics import model
from tbdynamics.inputs import fixed_parameters
from tbdynamics.utils import build_contact_matrix, round_sigfig
import plotly.graph_objects as go
#from emutools.parameters import load_param_info
#from emutools.calibration import round_sigfig
# from general_utils.doc_utils import TextElement, TableElement, FigElement, add_element_to_document, \
#     save_pyplot_add_to_doc, save_plotly_add_to_doc, compile_doc, generate_doc
from emutools.tex import StandardTexDoc

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
PROJECT_PATH = Path().resolve()
DATA_PATH = PROJECT_PATH / 'data'
SUPPLEMENT_PATH = PROJECT_PATH / 'supplement'

In [3]:
params = {
    "contact_rate": 0.009414102898074345,
    "start_population_size": 227344.75719536067,
    "progression_multiplier": 1.1,
    "rr_infection_latent": 0.2,
    "rr_infection_recovered": 0.2,
    'smear_positive_death_rate':0.364337776897486,
    'smear_negative_death_rate': 0.027588310343242016, 
    'smear_positive_self_recovery': 0.20344728302826143,
    'smear_negative_self_recovery': 0.22723824998716693,
}

In [4]:
pd.options.plotting.backend = "plotly"
time_start = 1800
time_end = 2020
time_step = 0.1

doc_sections = {}
compartments = [
    "susceptible",
    "early_latent",
    "late_latent",
    "infectious",
    "on_treatment",
    "recovered",
]
infectious_compartments = [
    "infectious",
    "on_treatment",
]

latent_compartments = [
    "early_latent",
    "late_latent",
]
age_strata = [0,5,15,35,50,70] 

In [5]:
app_doc = StandardTexDoc(PROJECT_PATH / 'supplement', 'supplement', "TB dynamics Supplement", 'tb')

In [6]:
matrix = build_contact_matrix()

In [7]:
matrix_homo = np.ones((6, 6))

In [8]:
matrix_homo

array([[1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.]])

In [9]:
tb_model = model.build_model(
    compartments,
    infectious_compartments,
    latent_compartments,
    age_strata,
    time_start,
    time_end,
    time_step,
    matrix, # change the matrix
    fixed_parameters,
    app_doc, 
    False
)

In [10]:
priors = [
    esp.UniformPrior("start_population_size", (1, 300000)),
    esp.UniformPrior("contact_rate", (0.0001, 0.02)),
    esp.UniformPrior("rr_infection_latent", (0.2, 0.5)),
    esp.UniformPrior("rr_infection_recovered", (0.1, 0.5)),
    esp.UniformPrior("smear_positive_death_rate", (0.335, 0.449)),
    esp.UniformPrior("smear_negative_death_rate", (0.017, 0.035)),
    esp.UniformPrior("smear_positive_self_recovery", (0.177, 0.288)),
    esp.UniformPrior("smear_negative_self_recovery", (0.073, 0.209)),
    # esp.UniformPrior("rr_progression_diabetes", (2.0, 10.0)),
    esp.UniformPrior("progression_multiplier", (0.1, 2.0)),
    # esp.UniformPrior("acf_scaleup_shape", (0.,0.1)),
    # # esp.UniformPrior("acf_inflection_time", (1980,2019)),
    # esp.UniformPrior("acf_start_asymp", (0.,1.)),
    # esp.UniformPrior("acf_end_asymp", (0.,10.)),
    # UniformPrior("cdr_adjustment", [0.6, 1.0]),
    esp.UniformPrior("infect_death_rate_dict.smear_positive", [0.335, 0.449]),
    esp.UniformPrior("infect_death_rate_dict.smear_negative", [0.017, 0.035]),
    esp.UniformPrior("self_recovery_rate_dict.smear_positive", [0.177, 0.288]),
    esp.UniformPrior("self_recovery_rate_dict.smear_negative", [0.073, 0.209]),
    # esp.UniformPrior("seed_time", [1800.0, 1960.0]),
    # esp.UniformPrior("seed_duration", [1.0, 100.0]),
    # esp.UniformPrior("seed_rate", [1.0, 100000.0]),
]
pop = pd.Series({2009: 1207100, 2019: 1194300})
notif = pd.Series({2011: 1495,2012: 1485,2013: 1369,2014:1405,2015:1642, 2016:1555, 2017:1440, 2018:1468, 2019:1417})
latent = pd.Series({2016:36})

targets = [
    est.NegativeBinomialTarget('total_population', pop, dispersion_param=2000),
    est.NormalTarget('percentage_latent', latent, stdev=5)
]
calibration_model = BayesianCompartmentalModel(tb_model, params, priors, targets)

In [11]:
with pm.Model() as pmc_model:
    start_params = {k: np.clip(v, *calibration_model.priors[k].bounds(0.99)) for k, v in params.items() if k in calibration_model.priors}
    variables = epm.use_model(calibration_model)
    map_params = pm.find_MAP(start=start_params, vars=variables, include_transformed=False)
    map_params = {k: float(v) for k, v in map_params.items()}
    print('Best calibration parameters found:')
for i_param, param in enumerate(map_params):
    print(f'   {param}: {round_sigfig(map_params[param], 4)} (within bound {priors[i_param].bounds()}')


Best calibration parameters found:
   start_population_size: 299000.0 (within bound (1.0, 300000.0)
   contact_rate: 0.01005 (within bound (0.0001, 0.02)
   rr_infection_latent: 0.4926 (within bound (0.2, 0.5)
   rr_infection_recovered: 0.1014 (within bound (0.1, 0.5)
   smear_positive_death_rate: 0.449 (within bound (0.335, 0.449)
   smear_negative_death_rate: 0.035 (within bound (0.017, 0.035)
   smear_positive_self_recovery: 0.288 (within bound (0.177, 0.288)
   smear_negative_self_recovery: 0.209 (within bound (0.073, 0.20900000000000002)
   progression_multiplier: 1.635 (within bound (0.1, 2.0)
   infect_death_rate_dict.smear_positive: 0.449 (within bound (0.335, 0.449)
   infect_death_rate_dict.smear_negative: 0.035 (within bound (0.017, 0.035)
   self_recovery_rate_dict.smear_positive: 0.288 (within bound (0.177, 0.288)
   self_recovery_rate_dict.smear_negative: 0.209 (within bound (0.073, 0.20900000000000002)


In [12]:
params.update(map_params)
tb_model.run(params)
derived_df_0 = tb_model.get_derived_outputs_df()
plots = {"total_population": {
      "title": "Population size",
      "output_key": "total_population",
      "times": [2009.0, 2019.0],
      "values": [1207100, 1194300],
      "quantiles": [0.025, 0.25, 0.5, 0.75, 0.975]
    },
     "notifications": {
      "title": "Notifications",
      "output_key": "notifications",
      "times": [2011.0, 2012.0, 2013.0, 2014.0, 2015.0, 2016.0, 2017.0, 2018.0, 2019.0],
      "values": [1495, 1485, 1369, 1405, 1642, 1555, 1440, 1468, 1417],
      "quantiles": [0.025, 0.25, 0.5, 0.75, 0.975]
    },
    "percentage_latent": {
      "title": "Percentage Latent",
      "output_key": "percentage_latent",
      "times": [2016.0],
      "values": [30.8],
      "quantiles": [0.025, 0.25, 0.5, 0.75, 0.975]
    },
    
    }

In [13]:
fig2_1 = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="total_population",
)
fig2_2 = px.scatter(x= plots['total_population']['times'], y = plots['total_population']['values'])
fig2_2.update_traces(marker=dict(color="red"))
fig2_3 = go.Figure(
    data=fig2_1.data + fig2_2.data,
)
fig2_3.update_layout(
    title="Modelled vs Data", title_x=0.5, xaxis_title="Year", yaxis_title="Population"
)
fig2_3.show()


In [14]:
#pop = derived_df_0[[f'total_populationXage_{i}' for i in age_strata]] 

In [15]:
tb_model.get_derived_outputs_df()

,infectious_population_size,latent_population_size,percentage_latent,prevalence_infectious,total_population,total_populationXage_0,total_populationXage_15,total_populationXage_35,total_populationXage_5,total_populationXage_50,total_populationXage_70
1800.0,1.000000,0.000000,0.000000,0.334449,298998.970803,49833.161800,49833.161800,49833.161800,49833.161800,49833.161800,49833.161800
1800.1,1.019203,3.418648,0.001142,0.340500,299325.085186,49843.174581,50044.068297,49696.332472,50307.094626,49801.280396,49633.134813
1800.2,1.127591,6.840386,0.002283,0.376297,299654.100073,49854.238574,50258.455523,49561.620075,50776.312720,49768.728239,49434.744942
1800.3,1.292435,10.409341,0.003470,0.430832,299986.006690,49866.339783,50476.256802,49429.025589,51240.885487,49735.524527,49237.974502
1800.4,1.500256,14.253165,0.004746,0.499551,300320.796060,49879.464683,50697.406415,49298.549539,51700.881281,49701.688206,49042.805935
...,...,...,...,...,...,...,...,...,...,...,...
2019.6,105.124011,1205.107326,81.514814,7110.706274,1478.390571,91.375353,312.606917,273.195998,122.342904,363.580121,315.289278
2019.7,104.786225,1201.532613,81.516928,7109.129727,1473.966990,91.091692,311.405978,272.274695,121.916617,362.637305,314.640703
2019.8,104.447501,1197.969111,81.519146,7107.421261,1469.555520,90.809210,310.210053,271.355765,121.492473,361.695523,313.992496
2019.9,104.111107,1194.413827,81.521267,7105.803002,1465.156110,90.527867,309.019125,270.439211,121.070467,360.754781,313.344658
